In [21]:
try:
    h2o.remove_all()
    h2o.shutdown()
except:
    pass

[WARNING] in <ipython-input-21-572c341a48bb> line 3:
    >>> h2o.shutdown()
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
H2O session _sid_883d closed.


In [2]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
import h2o
import numpy as np
import pandas as pd
from h2o.estimators.word2vec import H2OWord2vecEstimator
pd.options.display.max_columns= None
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from h2o.automl import H2OAutoML


In [3]:
h2o.init(min_mem_size='11G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/m/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp3gp0pmha
  JVM stdout: /tmp/tmp3gp0pmha/h2o_m_started_from_python.out
  JVM stderr: /tmp/tmp3gp0pmha/h2o_m_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,2 months and 18 days
H2O cluster name:,H2O_from_python_m_ghp425
H2O cluster total nodes:,1
H2O cluster free memory:,11 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [16]:
#load data

train_data=h2o.import_file('train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
train_data

qid,question_text,target
00002165364db923c7e6,How did Quebec nationalists see their province as a nation in the 1960s?,0
000032939017120e6e44,"Do you have an adopted dog, how would you encourage people to adopt and not shop?",0
0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity affect space geometry?,0
000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg hemispheres?,0
0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain bike by just changing the tyres?,0
00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?",0
00005059a06ee19e11ad,"Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?",0
0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off? Germs are everywhere.,0
00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?",0
00006e6928c5df60eacb,"Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. How did things change?",0


In [4]:
#load pretrained embeddings
gl=h2o.import_file('glove.840B.300d.txt')



Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
#get stop words
from nltk.corpus import stopwords
#nltk.download()
STOP_WORDS=list(stopwords.words('english'))
STOP_WORDS

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [20]:
#This converts sentenses into vectors
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words
words=tokenize(train_data['question_text'])

w2v_model = H2OWord2vecEstimator(pre_trained=gl)
w2v_model.train(training_frame=words)
survey= w2v_model.transform(words, aggregate_method = "AVERAGE")

word2vec Model Build progress: |█████████████████████████████████████████ (cancelled) 100%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_9ea404cc4448a69f74cd436af295485a> was cancelled by the user.